from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# function to remove stopwords
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)


movies_new['clean_plot'] = movies_new['clean_plot'].apply(lambda x: remove_stopwords(x))